# 🍎 Modelo Phi-4 con AIProjectClient 🍏

**Phi-4** es un modelo abierto de próxima generación que tiene como objetivo proporcionar capacidades cercanas a GPT-4 a una fracción del costo, haciéndolo ideal para muchos casos de uso empresariales o personales. Es especialmente excelente para chain-of-thought reasoning y escenarios de RAG (Retrieval Augmented Generation).

En este notebook, verás cómo:
1. **Inicializar** un `AIProjectClient` para tu entorno de Azure AI Foundry.
2. **Chatear** con el modelo **Phi-4** usando `azure-ai-inference`.
3. **Mostrar** un ejemplo de educación
4. **Disfrutar** de la propuesta de valor de una alternativa más económica a GPT-4, con fuertes capacidades de razonamiento. 🏋️

> **Disclaimer**: Esto no es un consejo médico. Por favor, consulta a profesionales.

## ¿Por qué Phi-4?
Phi-4 es un modelo de 14B parámetros entrenado en datos curados para un alto rendimiento en razonamiento.
- **Rentable**: Obtén rendimiento a nivel GPT-4 para muchas tareas sin el precio de GPT-4.
- **Razonamiento & RAG**: Perfecto para pasos de chain-of-thought reasoning y flujos de trabajo de retrieval augmented generation.
- **Ventana de contexto generosa**: 16K tokens, lo que permite más contexto o conversaciones de usuario más largas.


## 1. Setup

Importar la libreria
- **azure-ai-projects**: para el `AIProjectClient`.
- **azure-ai-inference**: para llamar los modelos
- **azure-identity**: para `DefaultAzureCredential`.

Proporciona un archivo `.env` con:
```bash
PROJECT_CONNECTION_STRING=<your-conn-string>
```

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage

from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '../.env', override=True)

conn_string = os.getenv("PROJECT_CONNECTION_STRING")
phi4_deployment = "phi-4"

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=conn_string,
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)

✅ AIProjectClient created successfully!


## 2. Chat with Phi-4 🍏
Demostraremos una conversación simple utilizando Phi-4 en un contexto de educación. Definiremos un mensaje del sistema que aclare el rol del asistente. Luego haremos algunas consultas de usuario.

> Nota: Phi-4 es muy adecuado para el razonamiento paso a paso. Permitiremos que ilustre sus pasos de razonamiento por diversión.



In [2]:
def chat_with_phi4(user_question, chain_of_thought=False):
    """Send a chat request to the Phi-4 model with optional chain-of-thought."""
    # We'll define a system message with disclaimers:
    system_prompt = (
        "Eres Virtual GPT, un asesor virtual amigable especializado en recomendar cursos virtuales para SENA.\n"
        "Recuerda siempre a los usuarios: No soy un asesor académico oficial.\n"
        "Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual."
    )

    # We can optionally instruct the model to show chain-of-thought. (Use carefully in production.)
    if chain_of_thought:
        system_prompt += "Por favor, muestra tu razonamiento paso a paso en tu respuesta.\n"

    # We create messages for system + user.
    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.8,  # a bit creative
            top_p=0.9,
            max_tokens=400,
        )

    return response.choices[0].message.content

# Example usage:
question = "Estoy buscando recomendaciones de cursos virtuales para enriquecer mi educación. ¿Podrías sugerirme algunos cursos y explicar brevemente de qué se tratan?"
answer = chat_with_phi4(question, chain_of_thought=True)
print("🗣️ User:", question)
print("🤖 Phi-4:", answer)

🗣️ User: Estoy buscando recomendaciones de cursos virtuales para enriquecer mi educación. ¿Podrías sugerirme algunos cursos y explicar brevemente de qué se tratan?
🤖 Phi-4: ¡Por supuesto! Aquí tienes algunas recomendaciones de cursos virtuales que podrían interesarte. Estos cursos están diseñados para cubrir una variedad de áreas y habilidades, permitiéndote elegir según tus intereses y necesidades de aprendizaje. Recuerda que no soy un asesor académico oficial, así que te animo a explorar cada uno y ver cuál se adapta mejor a tus objetivos.

1. **Programación y Tecnología de la Información:**
   - **Curso de Introducción a la Programación con Python:** Este curso ofrece una introducción al lenguaje de programación Python, una herramienta versátil utilizada en ciencia de datos, desarrollo web y automatización. Aprenderás conceptos básicos como variables, estructuras de control y funciones.
   - **Desarrollo Web Básico:** Ideal para aquellos interesados en crear sitios web. Este curso c

## 3. RAG-like Example (Stub)
Phi-4 también se destaca en escenarios de generación aumentada por recuperación, donde proporcionas contexto externo y permites que el modelo razone sobre ello. A continuación se muestra un ejemplo stub que ilustra cómo pasar el texto recuperado como contexto.

> En un escenario real, insertarías y buscarías pasajes relevantes, y luego los alimentarías en el mensaje de usuario/sistema.


In [3]:
def chat_with_phi4_rag(user_question, retrieved_doc):
    """Simulate an RAG flow by appending retrieved context to the system prompt."""
    system_prompt = (
        "Eres Virtual GPT, un asesor virtual amigable especializado en recomendar cursos virtuales.\n"
        "Recuerda siempre a los usuarios: No soy un asesor académico oficial.\n"
        "Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual.\n"
        "Tenemos algunos datos de la base de conocimientos del usuario: \n"
        f"{retrieved_doc}\n"
        "Por favor, utiliza este contexto para ayudar en tu respuesta. Si el contexto no ayuda, dilo.\n"
    )
    

    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.3,
            max_tokens=300,
        )
    return response.choices[0].message.content

# Let's define a dummy doc snippet:
doc_snippet = "Administracion de Recursos Humanos\n" \
              "Administracion de Recursos de M365\n" \
              "Acciones para conservación, protección y restauración de sistemas ecológicos\n" 	

user_q = "¿Que cursos ofreces relacionados con ecologia?"
rag_answer = chat_with_phi4_rag(user_q, doc_snippet)
print("🗣️ User:", user_q)
print("🤖 Phi-4 (RAG):", rag_answer)

🗣️ User: ¿Que cursos ofreces relacionados con ecologia?
🤖 Phi-4 (RAG): ¡Claro! Aquí tienes algunas recomendaciones de cursos relacionados con la ecología, basadas en el contexto que proporcionaste:

1. **Acciones para conservación, protección y restauración de sistemas ecológicos**:
   - Este curso probablemente se centra en estrategias prácticas y teóricas para la conservación y restauración de ecosistemas. Podrías aprender sobre la importancia de la biodiversidad, métodos de restauración ecológica y políticas de conservación.

2. **Cursos generales de ecología**:
   - Busca cursos que cubran temas como ecología de poblaciones, ecología de comunidades, ecología de ecosistemas y ecología global. Estos cursos te proporcionarán una base sólida en los principios fundamentales de la ecología.

3. **Cursos especializados en áreas específicas de la ecología**:
   - Considera cursos que se centren en áreas específicas como ecología marina, ecología forestal, ecología urbana o ecología de la c